<a href="https://colab.research.google.com/github/njagaty/Machine-leaning-Projects/blob/main/Stock_Price_Prediction_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('Om Sri Sai Ram')

Om Sri Sai Ram


Introduction : LSTM stands for Long Short Term Memory Networks. It is a type of recurrent neural network that is commonly used for regression and time series forecasting in machine learning. It can memorize data for long periods, which differentiates LSTM neural networks from other neural networks. I will start this task by importing all the necessary Python libraries and collecting the latest stock price data of Wipro:

In [5]:
! pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

In [7]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=5000)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('WIPRO.NS', 
                      start=start_date, 
                      end=end_date, 
                      progress=False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", 
             "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)
data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
3376,2023-01-16 00:00:00+05:30,390.000000,401.000000,389.000000,398.600006,398.600006,11933722
3377,2023-01-17 00:00:00+05:30,400.049988,402.049988,393.750000,396.350006,396.350006,7155016
3378,2023-01-18 00:00:00+05:30,398.500000,403.850006,398.000000,403.200012,403.200012,8040213
3379,2023-01-19 00:00:00+05:30,402.100006,404.000000,398.750000,403.399994,403.399994,4723728
3380,2023-01-20 00:00:00+05:30,403.549988,405.399994,401.299988,403.149994,403.149994,4632807


In [10]:
data.head(4)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-05-18 00:00:00+05:30,90.000008,103.488754,90.000008,93.813751,81.491493,177951
1,2009-05-19 00:00:00+05:30,94.803757,98.775002,85.522507,86.703751,75.315384,21478514
2,2009-05-20 00:00:00+05:30,87.750008,90.135002,84.498756,89.156258,77.445763,10974852
3,2009-05-21 00:00:00+05:30,88.425003,88.942505,83.992500,84.566254,73.458633,4682266


A candlestick chart gives a clear picture of the increase and decrease in stock prices, so let’s visualize a candlestick chart of the data before moving further:

In [13]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"], 
                                        high=data["High"],
                                        low=data["Low"], 
                                        close=data["Close"])])
figure.update_layout(title = "Stock Price Analysis by N Jagaty", 
                     xaxis_rangeslider_visible=False)
figure.show()

Now let’s have a look at the correlation of all the columns with the Close column as it is the target column:

In [14]:
correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

Close        1.000000
High         0.999797
Low          0.999778
Open         0.999500
Adj Close    0.999330
Volume       0.259793
Name: Close, dtype: float64


## **Training LSTM for Stock Price Prediction**

Now I will start with training an LSTM model for predicting stock prices. I will first split the data into training and test sets:

In [16]:
x = data[["Open", "High", "Low", "Volume"]]
y = data["Close"]
x = x.to_numpy()
y = y.to_numpy()
y = y.reshape(-1, 1)

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

Now I will prepare a neural network architecture for LSTM:

In [17]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (xtrain.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 128)            66560     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 25)                1625      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 117,619
Trainable params: 117,619
Non-trainable params: 0
_________________________________________________________________


Now here is how we can train our neural network model for stock price prediction:

In [18]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(xtrain, ytrain, batch_size=1, epochs=30)

Epoch 1/30
2704/2704 [==============================] - 19s 6ms/step - loss: 6907.8906
Epoch 2/30
2704/2704 [==============================] - 15s 6ms/step - loss: 576.2972
Epoch 3/30
2704/2704 [==============================] - 15s 6ms/step - loss: 425.4905
Epoch 4/30
2704/2704 [==============================] - 16s 6ms/step - loss: 401.6070
Epoch 5/30
2704/2704 [==============================] - 15s 6ms/step - loss: 465.5753
Epoch 6/30
2704/2704 [==============================] - 15s 6ms/step - loss: 437.9450
Epoch 7/30
2704/2704 [==============================] - 15s 6ms/step - loss: 536.6899
Epoch 8/30
2704/2704 [==============================] - 15s 6ms/step - loss: 364.8694
Epoch 9/30
2704/2704 [==============================] - 15s 5ms/step - loss: 342.6710
Epoch 10/30
2704/2704 [==============================] - 15s 6ms/step - loss: 372.4842
Epoch 11/30
2704/2704 [==============================] - 16s 6ms/step - loss: 451.1065
Epoch 12/30
2704/2704 [============================

Now let’s test this model by giving input values according to the features that we have used to train this model and predicting the final result:

In [25]:
import numpy as np
#features = [Open, High, Low, Adj Close, Volume]
features = np.array([[177.089996, 180.419998, 177.070007, 74919600]])
model.predict(features)

1/1 [==============================] - 1s 888ms/step


array([[185.17178]], dtype=float32)